In [1]:
%load_ext autoreload
%autoreload 2
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, diff_of_days, diff_of_times
from glob import glob
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
from train_prophet import main_prophet
import datetime
import math
import os
import numpy as np
import pandas as pd
import seaborn as sns
import sys
%matplotlib inline
sns.set()

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
               or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
               or path.count('R_email')
               or path.count('M')
               or path.count('Product')
              ]
test_paths  = [path for path in test_paths  
               if path.count('DT') 
               or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
               or path.count('R_email')
               or path.count('M')
               or path.count('Product')
              ]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)

In [15]:
START_DATE = '2017-11-04'
# START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
train_df['datetime'] = train_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
test_df['datetime'] = test_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
train_df['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
test_df['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
train_df['date'] = train_df['datetime'].map(lambda x: x.date())
test_df['date']  =  test_df['datetime'].map(lambda x: x.date())

In [16]:
list_regist = []
for d, diff in tqdm(train_df[['date', 'D1']].values):
    if diff < 999999:
        regist = date_add_days(d, -1*diff)
    else:
        regist = date_add_days(d, 0)
    list_regist.append(str(regist))

train_df['Regist_date'] = list_regist
    
list_regist = []
for d, diff in tqdm(test_df[['date', 'D1']].values):
    if diff < 999999:
        regist = date_add_days(d, -1*diff)
    else:
        regist = date_add_days(d, 0)
    list_regist.append(str(regist))

test_df['Regist_date'] = list_regist

100%|██████████| 506691/506691 [00:02<00:00, 248603.76it/s]


In [17]:
data = pd.concat([train_df, test_df], axis=0, ignore_index=True)

異なるuser keyで様々なユーザー特定パターン（正確でなくてもよい）を作り、それぞれのパターンで特徴を作る  
正確でなくてもよい理由としては、粒度を粗くしてもある条件を外した特徴量になるだけなので、問題ない

In [40]:
user_info = [
    'ProductCD',
    'Regist_date'
]
list_card = [
    'card1',
    'card2',
    'card3',
    'card4',
    'card5',
    'card6',
]
list_addr = [
    'addr1',
    'addr2',
]
list_pemail = [
    'P_emaildomain'
]
list_M = [col for col in data.columns if col.startswith('M')]

user_keys__card = user_info + list_card

user_keys__card_addr = user_info + list_card + list_addr
user_keys__card_pemail = user_info + list_card + list_pemail
user_keys__card_M = user_info + list_card + list_M

user_keys__card_addr_pemail = user_info + list_card + list_addr + list_pemail
user_keys__card_addr_M = user_info + list_card + list_addr + list_M
user_keys__card_pemail_M = user_info + list_card + list_pemail + list_M

user_keys__card_addr_pemail_M = user_info + list_card + list_addr + list_pemail + list_M

list_user_keys = [
  ['user_keys__card',      user_keys__card,],
  ['user_keys__card_addr',      user_keys__card_addr,],
  ['user_keys__card_pemail',      user_keys__card_pemail,],
  ['user_keys__card_M',      user_keys__card_M,],
  ['user_keys__card_addr_M',      user_keys__card_addr_M,],
  ['user_keys__card_addr_pemail',      user_keys__card_addr_pemail,],
  ['user_keys__card_pemail_M',      user_keys__card_pemail_M,],
  ['user_keys__card_addr_pemail_M',      user_keys__card_addr_pemail_M,],
]
check_Ds = ['D1', 'D2', 'D3', 'D4', 'D5', 'datetime']

data['D1'].fillna(-1, inplace=True)
data[user_info + list_card] = data[user_info + list_card].astype('str').fillna('#')

In [51]:
def create_new_user_id(data):
    
    uid = 'user_id'
    df = data.copy()

    for col in user_keys:
        df[col].fillna('#', inplace=True)
        
    df[user_keys] =  df[user_keys].astype('str')
    
    user_df = df.groupby(user_keys)[COLUMN_ID].min()
    user_df = user_df.to_frame(uid)
    user_df = df.set_index(user_keys).join(user_df)
    
    check_cols = [COLUMN_ID, uid, COLUMN_TARGET, 'Regist_date'] + check_Ds
    user_df = user_df.reset_index()[check_cols]
    user_df.sort_values(by=[uid, 'datetime'], inplace=True)
    
    col_same_user = f'same_{uid}'
    
    
    if uid not in user_df.columns:
        user_df.reset_index(inplace=True)
    
    col_same_user = f'same_{uid}'
    max_fraud = user_df.groupby(uid)[COLUMN_TARGET].max()
    if uid in user_df.columns:
        user_df.set_index(uid, inplace=True)
    user_df['max_Fraud'] = max_fraud
    
    # Fraudを持つ可能性のあるユーザーのみで考える
    user_df.reset_index(inplace=True)
    user_df['some_fraud_user'] = (user_df[COLUMN_TARGET] != user_df['max_Fraud'])*1
#     diff_fraud_users = user_df[user_df[COLUMN_TARGET] != user_df['max_Fraud']].reset_index()[uid].unique()
#     user_df = user_df[user_df[uid].isin(diff_fraud_users)]
    user_df.sort_values(by=[uid, 'datetime'], inplace=True)
    
    user_df['date'] = user_df['datetime'].map(lambda x: x.date())
    user_df['diff_from_regist'] = user_df['datetime'] - user_df['Regist_date'].map(lambda x: datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10]) ) )
    user_df['diff_from_regist'] = user_df['diff_from_regist'].map(lambda x: x.days)
    
    user_df['Regist_date_add_D1'] = user_df[['Regist_date', 'D1']].apply(lambda x: date_add_days(x[0], x[1]), axis=1)
    user_df['Regist_date_add_D1-D3'] = user_df[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]), axis=1)
    user_df['Regist_date_add_D1-D3-1'] = user_df[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]-1), axis=1)
    
    user_df['date__before1'] = user_df.groupby(uid)['date'].shift(1)
    user_df['date__before2'] = user_df.groupby(uid)['date'].shift(2)
    user_df['date__before3'] = user_df.groupby(uid)['date'].shift(3)
    user_df['D1__before1']   = user_df.groupby(uid)['D1'].shift(1)
    user_df['D1__before2']   = user_df.groupby(uid)['D1'].shift(2)
    
    user_df['diff_D1__before1'] = user_df[['D1', 'D1__before1']].apply(lambda x: x[0] - x[1] , axis=1)
    user_df['diff_D1__before2'] = user_df[['D1', 'D1__before2']].apply(lambda x: x[0] - x[1] , axis=1)
    user_df['diff_day__before1'] = user_df[['date', 'date__before1']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
    user_df['diff_day__before2'] = user_df[['date', 'date__before2']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
    user_df['diff_day__before3'] = user_df[['date', 'date__before3']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
    
    # ユーザーIDの作成
    cnt = 0
    p_cnt = 0
    list_user = []
    for user, d3, b1, b2, b3, D1_b1, D1_b2 in user_df[[uid, 'D3', 'diff_day__before1', 'diff_day__before2', 'diff_day__before3', 'diff_D1__before1', 'diff_D1__before2']].values:
        p_cnt+=1
#         if d3==b3 or d3==b3-1:
        if d3==b3:
            current_user = b3_user
#         elif d3==b2 or d3==b2-1:
        elif d3==b2:
            current_user = b2_user
#         elif d3==b1 or d3==b1-1:
        elif d3==b1:
            current_user = b1_user
#         elif D1_b1 == b1 and D1_b1 != 0:
#             current_user = b1_user
#         elif D1_b2 == b2 and D1_b2 != 0:
#             current_user = b2_user
        else:
            cnt += 1
        current_user = cnt
            
        if p_cnt>=3:
            b3_user = b2_user
        if p_cnt>=2:
            b2_user = b1_user
        b1_user = cnt
        list_user.append(current_user)
        
    user_df[col_same_user] = list_user
    print(f"Extract Same User ID.", user_df.shape)
    user_df = user_df[[COLUMN_ID, col_same_user, uid]]
    
    return user_df

In [52]:
for pattern, user_keys in tqdm(list_user_keys):
    df_new_user = create_new_user_id(data)
    df_new_user.to_csv(f'../output/0830_ieee__same_user__pattern-{pattern}.csv', index=False)



  0%|          | 0/8 [00:00<?, ?it/s]

Extract Same User ID. (1097231, 28)




 12%|█▎        | 1/8 [08:55<1:02:30, 535.83s/it]

Extract Same User ID. (1097231, 28)




 25%|██▌       | 2/8 [18:00<53:51, 538.58s/it]  

...



 75%|███████▌  | 6/8 [1:08:55<21:11, 635.59s/it]

Extract Same User ID. (1097231, 28)




 88%|████████▊ | 7/8 [1:18:06<10:10, 610.11s/it]

Extract Same User ID. (1097231, 28)




100%|██████████| 8/8 [1:29:58<00:00, 640.75s/it]

